# Exercícios de Pandas (com seus datasets)

Objetivo: você responder as questões **usando comandos da apostila** (Series, DataFrame, colunas, `iloc`/filtros, índices, dados ausentes, `merge/concat/join`, operações, `pivot_table`, séries temporais e I/O).

Datasets incluídos (dos ZIPs que você enviou):
- `gender_classification_v7.csv` (features binárias/numéricas + `gender`)
- `sp-historical.csv` (S&P 500 histórico)
- `spy-historical.csv` (SPY histórico)
- `all_stocks_5yr.csv` + pasta `individual_stocks_5yr/` (ações do S&P 500 por ~5 anos)

Regras deste notebook:
1. Cada questão tem uma célula de código vazia para você resolver.
2. Use pandas; evite bibliotecas extras.
3. Se travar, escreva sua tentativa e o erro, e eu corrijo em cima do seu código.


In [1]:
# Setup
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 140)
pd.set_option("display.max_rows", 80)

print("pandas:", pd.__version__)


pandas: 2.2.2


## 0) Caminhos e carregamento automático

Este notebook tenta achar os CSVs na pasta atual e subpastas.
Sugestão:
- Extraia seus ZIPs em uma pasta `data/` (ou rode a célula de extração abaixo).


In [2]:
from pathlib import Path
import os, zipfile

BASE_DIR = Path(".").resolve()
DATA_DIR = BASE_DIR / "data"
DATA_DIR.mkdir(exist_ok=True)

def find_file(filename: str, base: Path = BASE_DIR) -> Path:
    matches = list(base.rglob(filename))
    if not matches:
        raise FileNotFoundError(f"Não encontrei {filename}. Coloque o arquivo na pasta do notebook ou em subpastas.")
    # se houver mais de um, pega o primeiro
    return matches[0]

def maybe_extract_zips(base: Path = BASE_DIR, out_dir: Path = DATA_DIR):
    zips = sorted(base.glob("*.zip"))
    if not zips:
        print("Nenhum .zip encontrado na pasta do notebook. Se você já extraiu, pode ignorar.")
        return
    for zp in zips:
        try:
            with zipfile.ZipFile(zp, "r") as z:
                z.extractall(out_dir)
            print(f"OK: {zp.name} -> {out_dir}")
        except zipfile.BadZipFile:
            print(f"ERRO: zip inválido: {zp.name}")

maybe_extract_zips()
print("BASE_DIR:", BASE_DIR)
print("DATA_DIR:", DATA_DIR)


Nenhum .zip encontrado na pasta do notebook. Se você já extraiu, pode ignorar.
BASE_DIR: C:\Users\Filip\OneDrive\Área de Trabalho\SQL_Pandas
DATA_DIR: C:\Users\Filip\OneDrive\Área de Trabalho\SQL_Pandas\data


In [4]:
# Carregar datasets principais
path_gender = find_file("gender_classification_v7.csv")
path_sp     = find_file("sp-historical.csv")
path_spy    = find_file("spy-historical.csv")
path_all    = find_file("all_stocks_5yr.csv")

df_gender = pd.read_csv(path_gender)
df_sp     = pd.read_csv(path_sp)
df_spy    = pd.read_csv(path_spy)
df_all    = pd.read_csv(path_all)

print("df_gender:", df_gender.shape, "->", path_gender)
print("df_sp    :", df_sp.shape, "->", path_sp)
print("df_spy   :", df_spy.shape, "->", path_spy)
print("df_all   :", df_all.shape, "->", path_all)


df_gender: (5001, 8) -> C:\Users\Filip\OneDrive\Área de Trabalho\SQL_Pandas\Kaggle_datasets\archive (5)\gender_classification_v7.csv
df_sp    : (8990, 7) -> C:\Users\Filip\OneDrive\Área de Trabalho\SQL_Pandas\Kaggle_datasets\archive (3)\sp-historical.csv
df_spy   : (8210, 10) -> C:\Users\Filip\OneDrive\Área de Trabalho\SQL_Pandas\Kaggle_datasets\archive (3)\spy-historical.csv
df_all   : (619040, 7) -> C:\Users\Filip\OneDrive\Área de Trabalho\SQL_Pandas\Kaggle_datasets\archive (2)\all_stocks_5yr.csv


## 0.1) Checagem rápida (opcional)

Use para lembrar rapidamente colunas e tipos.


In [5]:
df_gender.head()

,long_hair,forehead_width_cm,forehead_height_cm,nose_wide,nose_long,lips_thin,distance_nose_to_lip_long,gender
0,1,11.8,6.1,1,0,1,1,Male
1,0,14.0,5.4,0,0,1,0,Female
2,0,11.8,6.3,1,1,1,1,Male
3,0,14.4,6.1,0,1,1,1,Male
4,1,13.5,5.9,0,0,0,0,Female


In [6]:
df_sp.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1990-01-02,353.40,359.69,351.98,359.69,359.69,162070000
1,1990-01-03,359.69,360.59,357.89,358.76,358.76,192330000
2,1990-01-04,358.76,358.76,352.89,355.67,355.67,177000000
3,1990-01-05,355.67,355.67,351.35,352.20,352.20,158530000
4,1990-01-08,352.20,354.24,350.54,353.79,353.79,140110000


In [7]:
df_spy.head()

,date_id,Date,Open,High,Low,Close,Adj Close,Volume,Dividend,forward_returns
0,780,1993-02-01,43.97,44.25,43.97,44.25,24.49,480500.0,0.0,0.002034
1,781,1993-02-02,44.22,44.38,44.13,44.34,24.54,201300.0,0.0,0.010600
2,782,1993-02-03,44.41,44.84,44.38,44.81,24.80,529400.0,0.0,0.004240
3,783,1993-02-04,44.97,45.09,44.47,45.00,24.90,531500.0,0.0,-0.000667
4,784,1993-02-05,44.97,45.06,44.72,44.97,24.88,492100.0,0.0,0.000000


In [ ]:
df_all.head()

# Parte A - `gender_classification_v7.csv`

Colunas:
- `long_hair`, `nose_wide`, `nose_long`, `lips_thin`, `distance_nose_to_lip_long` (0/1)
- `forehead_width_cm`, `forehead_height_cm` (numéricas)
- `gender` (Male/Female)


## Questão A1
Mostre `shape`, `columns`, `head(5)` e `info()` do dataset.

In [ ]:
# seu código aqui


## Questão A2
Use `describe()` e responda: quais colunas parecem binárias? (Dica: observe min/max e contagem).

In [ ]:
# seu código aqui


## Questão A3
Use `value_counts()` para contar quantas linhas há de `Male` e `Female`. Calcule também as proporções (%).

In [ ]:
# seu código aqui


## Questão A4
Crie um filtro: apenas `Female` com `long_hair == 1`. Quantas linhas existem?

In [ ]:
# seu código aqui


## Questão A5
Use `iloc` para selecionar linhas 10 a 19 e colunas 0 a 3. Depois, faça o mesmo usando `loc` (com nomes de colunas).

In [ ]:
# seu código aqui


## Questão A6
Crie uma coluna `forehead_ratio = forehead_width_cm / forehead_height_cm`. Mostre `describe()` dessa nova coluna.

In [ ]:
# seu código aqui


## Questão A7
Crie uma coluna categórica `ratio_cat` com `apply` + `lambda`: 'alta' se `forehead_ratio >= 2.2`, senão 'baixa'. Use `value_counts()` para contar.

In [ ]:
# seu código aqui


## Questão A8
Faça `groupby('gender')` e calcule a média de todas as colunas numéricas. Ordene o resultado pela média de `forehead_width_cm` (desc).

In [ ]:
# seu código aqui


## Questão A9
Use `groupby('gender').agg(...)` para obter `mean`, `std`, `min`, `max` de `forehead_width_cm` e `forehead_height_cm`.

In [ ]:
# seu código aqui


## Questão A10
Combine filtros: `Female` com `forehead_width_cm > 13` e `nose_wide == 1`. Quantas linhas? Qual a média de `forehead_height_cm` nesse subconjunto?

In [ ]:
# seu código aqui


## Questão A11
Encontre a linha com maior `forehead_width_cm` usando `idxmax()`. Mostre essa linha inteira.

In [ ]:
# seu código aqui


## Questão A12
Ordene o dataset por `forehead_ratio` (desc) e pegue as 10 maiores linhas com as colunas: `gender`, `forehead_width_cm`, `forehead_height_cm`, `forehead_ratio`.

In [ ]:
# seu código aqui


# Parte B - `sp-historical.csv` e `spy-historical.csv` (séries temporais)

Objetivo: praticar `to_datetime`, seleção por datas, índices e operações básicas.

Observações:
- `df_sp` tem `Date` (texto) e colunas OHLC + Volume.
- `df_spy` tem `Date` e também `Dividend` e `forward_returns` (há pelo menos 1 NaN).


## Preparação (faça uma vez antes das questões)

In [ ]:
# Converta as colunas de data para datetime e crie versões com índice de data (sem perder as originais)
# Dica: use pd.to_datetime(...)

# df_sp2 = ...
# df_spy2 = ...

# Se quiser, use set_index("Date") para facilitar seleção por intervalo:
# df_sp2 = df_sp2.set_index("Date")
# df_spy2 = df_spy2.set_index("Date")

# No final, deixe df_sp2 e df_spy2 disponíveis.


## Questão B1
No `df_sp2`, selecione apenas o intervalo de 2008-01-01 até 2009-12-31. Mostre `shape` e `head()`.

In [ ]:
# seu código aqui


## Questão B2
No intervalo selecionado (2008-2009), encontre o dia com maior `Volume` e mostre a data e o valor (use `idxmax`).

In [ ]:
# seu código aqui


## Questão B3
Crie uma coluna `retorno_close` em `df_sp2` com o retorno diário do `Close` (use `pct_change`). Mostre as 5 primeiras linhas onde o retorno não é NaN.

In [ ]:
# seu código aqui


## Questão B4
Crie uma coluna `year` (ano) a partir da data e faça `groupby('year')` para calcular o `max()` de `Close`. Qual ano teve o maior `Close`?

In [ ]:
# seu código aqui


## Questão B5
No `df_spy2`, verifique valores ausentes com `isna().sum()`. Em seguida, faça um `fillna(0)` em `forward_returns` e confirme que zerou NaNs.

In [ ]:
# seu código aqui


## Questão B6
No `df_spy2`, filtre apenas linhas com `Dividend > 0`. Quantas linhas existem? Mostre as 10 primeiras datas com dividendos.

In [ ]:
# seu código aqui


## Questão B7
Ordene `df_spy2` por `forward_returns` (desc) e mostre as 10 maiores linhas apenas com `Open`, `Close`, `forward_returns`.

In [ ]:
# seu código aqui


## Questão B8
Crie um resumo com `describe()` apenas para as colunas numéricas de `df_spy2`. Compare com `df_sp2` (qual tem maior dispersão em `Volume`?).

In [ ]:
# seu código aqui


# Parte C - Merge, Join e Concat (SP vs SPY)

Objetivo: praticar `merge`, `join` e comparar séries.

Sugestão:
- Use `merge` por coluna (`on='Date'`) ou `join` por índice (se você setou índice de data).


## Questão C1
Faça um `merge` entre `df_sp` e `df_spy` pela coluna `Date` (inner). Mostre `shape` e `head()` do resultado.

In [ ]:
# seu código aqui


## Questão C2
Após o merge, crie uma coluna `diff_close = Close_sp - Close_spy` (ajuste nomes conforme o pandas criou). Mostre estatísticas (`describe`) de `diff_close`.

In [ ]:
# seu código aqui


## Questão C3
Repita o merge com `how='outer'`. Quantas linhas a mais aparecem em relação ao inner? (Dica: compare shapes).

In [ ]:
# seu código aqui


## Questão C4
Se você criou `df_sp2` e `df_spy2` com índice de data, faça um `join` (left) de SP com SPY. Verifique se o resultado bate com o merge (em quantidade de linhas).

In [ ]:
# seu código aqui


## Questão C5
No dataset combinado, filtre um intervalo de datas (ex.: 2015-01-01 a 2016-12-31) e calcule a correlação entre os fechamentos (`Close`).

In [ ]:
# seu código aqui


# Parte D - `all_stocks_5yr.csv` (ações por ~5 anos)

Colunas:
- `date`, `open`, `high`, `low`, `close`, `volume`, `Name` (ticker)

Aqui você pratica: `unique/nunique`, `value_counts`, filtros, `groupby`, dados ausentes, `pivot_table`, índices multiníveis e exportação.


## Preparação (faça uma vez antes das questões)

In [ ]:
# Converta a coluna 'date' para datetime e crie df_all2
# df_all2 = ...
# (Opcional) df_all2 = df_all2.sort_values(["Name", "date"])


## Questão D1
Mostre `info()` e `memory_usage(deep=True)` do `df_all2`. Qual coluna parece consumir mais memória?

In [ ]:
# seu código aqui


## Questão D2
Quantos tickers (valores distintos) existem em `Name`? (use `nunique`).

In [ ]:
# seu código aqui


## Questão D3
Use `value_counts()` para listar os 10 tickers com mais linhas.

In [ ]:
# seu código aqui


## Questão D4
Filtre apenas o ticker `AAPL` e mostre o intervalo mínimo e máximo de datas disponíveis (min/max em `date`).

In [ ]:
# seu código aqui


## Questão D5
Para o ticker `AAPL`, encontre o dia com maior `close` (use `idxmax` + seleção da linha).

In [ ]:
# seu código aqui


## Questão D6
Faça `groupby('Name')` e calcule `mean()` de `close`. Mostre os 10 maiores tickers pela média de `close`.

In [ ]:
# seu código aqui


## Questão D7
Faça `groupby('Name').agg(...)` para obter: `max` de `high`, `min` de `low` e `sum` de `volume`. Mostre os 10 maiores por `sum(volume)`.

In [ ]:
# seu código aqui


## Questão D8
Verifique valores ausentes em `open/high/low`. Depois, crie uma versão sem NaNs nessas colunas (use `dropna(subset=...)`). Compare quantas linhas foram removidas.

In [ ]:
# seu código aqui


## Questão D9
Crie uma versão preenchida: preencha `open/high/low` faltantes com o valor de `close` da mesma linha (`fillna`). Confirme que zerou os NaNs nessas colunas.

In [ ]:
# seu código aqui


## Questão D10
Crie um índice multinível com `set_index(['Name','date'])`. Depois use `.loc` para selecionar apenas `('MSFT', slice('2014-01-01','2014-12-31'))` e mostre `head()`.

In [ ]:
# seu código aqui


## Questão D11
Crie uma `pivot_table` para comparar preços de fechamento de 4 tickers: `['AAPL','MSFT','AMZN','GOOGL']` com `index='date'`, `columns='Name'`, `values='close'`, `aggfunc='mean'`. Mostre `head()`.

In [ ]:
# seu código aqui


## Questão D12
Com a tabela dinâmica criada, calcule a correlação entre as colunas (tickers).

In [ ]:
# seu código aqui


## Questão D13
Mapeamento: crie uma coluna `grupo` em `df_all2` que marque `AAPL/MSFT/AMZN/GOOGL` como 'big_tech' e o restante como 'outros' (use `map` ou `apply`). Depois faça `value_counts()` de `grupo`.

In [ ]:
# seu código aqui


## Questão D14
Exportação: gere um DataFrame com os 20 maiores tickers por `mean(close)` e salve em CSV com `to_csv('top20_mean_close.csv', index=False)`.

In [ ]:
# seu código aqui


# Parte E (Opcional) - Concat com arquivos individuais (pasta `individual_stocks_5yr/`)

Objetivo: praticar `concat` carregando 3 CSVs individuais e comparando com `df_all2`.

Tickers sugeridos: `AAPL`, `MSFT`, `AMZN` (os arquivos existem na pasta do zip).


In [ ]:
# Encontre a pasta de individuais e defina paths para 3 tickers
# Dica: use find_file("AAPL_data.csv") etc., ou rglob dentro de uma pasta.

# path_aapl = ...
# path_msft = ...
# path_amzn = ...

# df_aapl = pd.read_csv(path_aapl)
# df_msft = pd.read_csv(path_msft)
# df_amzn = pd.read_csv(path_amzn)

# df_3 = pd.concat([df_aapl, df_msft, df_amzn], axis=0, ignore_index=True)
# df_3.head()


## Questão E1
Depois de criar `df_3` (concat dos 3 tickers), compare `df_3.shape` com um filtro equivalente em `df_all2` (apenas esses 3 tickers). Eles têm o mesmo número de linhas?

In [ ]:
# seu código aqui


## Questão E2
Ordene `df_3` por `['Name','date']` e crie um índice multinível (`set_index`). Selecione apenas `AMZN` e mostre o ano de 2016.

In [ ]:
# seu código aqui


## Questão E3
Crie uma `pivot_table` em `df_3` igual à da questão D11 e compare com a pivot criada em D11 (apenas para as datas que existirem).

In [ ]:
# seu código aqui


# Área livre

Use este espaço para:
- tentar suas soluções antes de “passar a limpo”,
- criar resumos,
- salvar resultados intermediários.


In [ ]:
# rascunho
